# conda

In [ ]:
import copy
from collections import defaultdict
from pathlib import Path

import doit
import importnb
import yaml
from packaging.utils import canonicalize_name

with importnb.Notebook():
    import jupyak.tasks._actions as A
    import jupyak.tasks._py as P
    from jupyak.tasks._yak import Yak

In [ ]:
def conda_tasks(yak: Yak):
    env = yak.env
    yml_dep = [
        P.get_dep_file(py_repo.work_path / ppt, yak)
        for py_repo in yak.py_repos.values()
        for ppt in py_repo.py.pyproject_tomls
    ]
    yml = env.environment_yml

    yield dict(
        name="environment.yml",
        uptodate=[
            doit.tools.config_changed(env.environment_yml_data),
        ],
        actions=[
            (doit.tools.create_folder, [yak.work_path]),
            (make_env, [yak, yml_dep]),
        ],
        targets=[yml],
        file_dep=yml_dep,
    )

    mamba_args = ["mamba", "env", "update", "--prefix", env.venv, "--file", yml]

    yield dict(
        name="update",
        actions=[A.run([*mamba_args], {"cwd": yak.work_path})],
        file_dep=[yml],
        targets=[env.venv_history],
    )

In [ ]:
def make_env(yak: Yak, dep_files: list[Path]):
    env = copy.deepcopy(yak.env.environment_yml_data)

    dep_specs = defaultdict(list)

    norm_repos = {canonicalize_name(dep_file.stem) for dep_file in dep_files}

    for dep_file in dep_files:
        deps = yaml.safe_load(dep_file.read_text(encoding="utf-8"))["dependencies"]
        skipped = []
        for dep, spec in deps.items():
            if dep in norm_repos:
                skipped += [dep]
                continue
            dep_specs[dep].append(spec)
        if skipped:
            print(
                f"""   ... {dep_file.stem} will use editable: {", ".join(sorted(set(skipped)))}""",
            )
    for dep, specs in dep_specs.items():
        spec = ",".join(s for s in sorted(set(specs)) if s.strip())
        env["dependencies"] += [
            f"""{yak.env.pypi_to_conda.get(dep, dep)} {spec}""".strip(),
        ]
    env["dependencies"] = sorted(env["dependencies"])
    yaml_text = yaml.safe_dump(env, default_flow_style=False)
    yak.env.environment_yml.write_text(yaml_text, encoding="utf-8")